- Time to concatenate 24 hours of LFP (12x 2hr files) = 30 minutes. 
- Size of 24 hours of LFP = ~160 GB

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from joblib import Parallel, delayed

from ecephys_analyses.prepro import get_run_specs, run_catgt

In [ ]:
# ----------
# USER

####
######## Manual
# Subject, experiment, run, catgt_cfg
# catGT_cfg must contains the following keys: run, t, g, prb, 
# all_run_specs = [
#     ('CNPIX3-Valentino', '2-19-2020', '2-19-2020',    {'g': '1', 'prb': '0', 't': '4,4',}), 
# ]
########## Auto generation of bins to process

# [(<subject>, <condition_group>), ]
condition_groups = [
    ('Doppio', 'SD_catgt'),
    ('Allan', 'SD_catgt'),
    ('Luigi', 'SD_catgt'),
    ('Charles', 'SD_catgt'),
]

rerun_existing=False  # Pass if output meta file exists

all_run_specs = []
for subject, condition_group in condition_groups:
    all_run_specs += get_run_specs(subject, condition_group, rerun_existing=rerun_existing)
print(f"N runs (rerun_existing={rerun_existing}) = {len(all_run_specs)}")
print(f"run specs (first 5). ```subject, experiment, run, catgt_cfg``` Looks good?: {all_run_specs[0:5]}")

####
####
####

n_jobs = 1

dry_run = False

# END USER
# ----------

In [ ]:
# Keys in roots.yml
src_root_key = 'raw_chronic'
tgt_root_key = 'catgt'

In [ ]:
if n_jobs == 1:
    for run_specs in all_run_specs:
        print(run_specs)
        run_catgt(run_specs, dry_run=dry_run, src_root_key=src_root_key, tgt_root_key=tgt_root_key)
else:
    assert False
#     parallel = Parallel(
#         n_jobs=n_jobs,
#         backend='multiprocessing',
#     )(delayed(run_catgt)(run_specs, dry_run=dry_run) for run_specs in all_run_specs)